# Hypothetical Prompt Embeddings (HyPE)

## 개요

이 코드는 Hypothetical Prompt Embeddings (HyPE)로 향상된 Retrieval-Augmented Generation (RAG) 시스템을 구현합니다. 쿼리-문서 스타일 불일치로 어려움을 겪는 기존 RAG 파이프라인과 달리, HyPE는 인덱싱 단계에서 가상 질문을 미리 계산합니다. 이를 통해 검색을 질문-질문 매칭 문제로 변환하여 비용이 많이 드는 런타임 쿼리 확장 기법이 필요 없어집니다.

## 노트북의 주요 구성 요소

1. PDF 처리 및 텍스트 추출
2. 일관된 정보 단위를 유지하기 위한 텍스트 청킹
3. LLM을 사용한 **가상 프롬프트 임베딩 생성** - 청크당 여러 프록시 질문 생성
4. [FAISS](https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/) 및 OpenAI 임베딩을 사용한 벡터 스토어 생성
5. 처리된 문서를 쿼리하기 위한 검색기 설정
6. RAG 시스템 평가

## 방법 세부사항

### 문서 전처리

1. PDF는 `PyPDFLoader`를 사용하여 로드됩니다.
2. 텍스트는 지정된 청크 크기와 겹침을 사용하여 `RecursiveCharacterTextSplitter`로 청크로 분할됩니다.

### 가상 질문 생성

원시 텍스트 청크를 임베딩하는 대신, HyPE는 각 청크에 대해 **여러 가상 프롬프트를 생성**합니다. 이러한 **미리 계산된 질문**은 사용자 쿼리를 시뮬레이션하여 실제 검색과의 정렬을 개선합니다. 이를 통해 HyDE와 같은 기법에서 필요로 하는 런타임 합성 답변 생성이 필요 없어집니다.

### 벡터 스토어 생성

1. 각 가상 질문은 OpenAI 임베딩을 사용하여 임베딩됩니다.
2. **각 질문 임베딩을 원본 청크와 연결**하여 FAISS 벡터 스토어를 구축합니다.
3. 이 접근 방식은 **청크당 여러 표현을 저장**하여 검색 유연성을 증가시킵니다.

### 검색기 설정

1. 검색기는 직접 문서 검색이 아닌 **질문-질문 매칭**에 최적화됩니다.
2. FAISS 인덱스는 가상 프롬프트 임베딩에 대한 **효율적인 최근접 이웃** 검색을 가능하게 합니다.
3. 검색된 청크는 다운스트림 LLM 생성에 **더 풍부하고 정확한 컨텍스트**를 제공합니다.

## 주요 기능

1. **미리 계산된 가상 프롬프트** – 런타임 오버헤드 없이 쿼리 정렬을 개선합니다.
2. **다중 벡터 표현** – 각 청크는 더 넓은 의미적 커버리지를 위해 여러 번 인덱싱됩니다.
3. **효율적인 검색** – FAISS는 향상된 임베딩에 대한 빠른 유사도 검색을 보장합니다.
4. **모듈화된 설계** – 파이프라인은 다양한 데이터셋 및 검색 설정에 쉽게 적응할 수 있습니다. 또한 재순위 지정과 같은 대부분의 최적화와 호환됩니다.

## 평가

HyPE의 효과는 여러 데이터셋에 걸쳐 평가되었으며, 다음을 보여줍니다:

- 검색 정밀도에서 최대 42% 포인트 개선
- 클레임 재현율에서 최대 45% 포인트 개선
    (전체 평가 결과는 [사전 인쇄본](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=5139335) 참조)

## 이 접근 방식의 이점

1. **쿼리 타임 오버헤드 제거** – 모든 가상 생성은 인덱싱 시 오프라인으로 수행됩니다.
2. **향상된 검색 정밀도** – 쿼리와 저장된 콘텐츠 간의 더 나은 정렬.
3. **확장 가능하고 효율적** – 쿼리당 추가 계산 비용 없음; 검색은 표준 RAG만큼 빠릅니다.
4. **유연하고 확장 가능** – 재순위 지정과 같은 고급 RAG 기법과 결합할 수 있습니다.

## 결론

HyPE는 쿼리-문서 스타일 불일치를 극복하면서 런타임 쿼리 확장의 계산 비용을 피하여 기존 RAG 시스템에 대한 확장 가능하고 효율적인 대안을 제공합니다. 가상 프롬프트 생성을 인덱싱으로 이동함으로써 검색 정밀도와 효율성을 크게 향상시켜 실제 응용 프로그램을 위한 실용적인 솔루션을 만듭니다.

자세한 내용은 전체 논문을 참조하세요: [사전 인쇄본](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=5139335)


<div style="text-align: center;">

<img src="../images/hype.svg" alt="HyPE" style="width:70%; height:auto;">
</div>

# 패키지 설치 및 임포트

아래 셀은 이 노트북을 실행하는 데 필요한 모든 패키지를 설치합니다.


In [ ]:
# Install required packages
# !pip install faiss-cpu futures langchain-community python-dotenv tqdm
!pip install -q faiss-cpu futures python-dotenv tqdm langchain langchain-community langchain-openai langchain-text-splitters langchain-core pypdf PyMuPDF pydantic>=2.0 rank-bm25 openai tiktoken deepeval

In [ ]:
# 도우미 함수 및 평가 모듈에 액세스하기 위해 저장소 복제
!git clone https://github.com/please123/RAG_TECHNIQUES.git
import sys
sys.path.append('RAG_TECHNIQUES')
# 최신 데이터로 실행하려면
# !cp -r RAG_TECHNIQUES/data .

In [ ]:
import os
import sys
import faiss
from tqdm import tqdm
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor, as_completed
from langchain_community.docstore.in_memory import InMemoryDocstore


# .env 파일에서 환경 변수 로드
load_dotenv()

# OpenAI API 키 환경 변수 설정 (OpenAI를 사용하지 않는 경우 주석 처리)
if not os.getenv('OPENAI_API_KEY'):
    os.environ["OPENAI_API_KEY"] = input("Please enter your OpenAI API key: ")
else:
    os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

# Colab 호환성을 위해 원본 경로 추가가 대체됨
from helper_functions import *
from evaluation.evalute_rag import *


### 상수 정의

- `PATH`: RAG 파이프라인에 임베딩될 데이터의 경로

이 튜토리얼은 OpenAI 엔드포인트를 사용합니다 ([사용 가능한 모델](https://platform.openai.com/docs/pricing)). 
- `LANGUAGE_MODEL_NAME`: 사용할 언어 모델의 이름. 
- `EMBEDDING_MODEL_NAME`: 사용할 임베딩 모델의 이름.

이 튜토리얼은 청킹 길이 함수로 python `len` 함수를 사용하는 `RecursiveCharacterTextSplitter` 청킹 접근 방식을 사용합니다. 여기서 조정할 청킹 변수는 다음과 같습니다:
- `CHUNK_SIZE`: 하나의 청크의 최소 길이
- `CHUNK_OVERLAP`: 연속된 두 청크 간의 겹침.

In [ ]:
# 필요한 데이터 파일 다운로드
import os
os.makedirs('data', exist_ok=True)

# 이 노트북에서 사용할 PDF 문서 다운로드
!wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf
!wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf


In [64]:
PATH = "data/Understanding_Climate_Change.pdf"
LANGUAGE_MODEL_NAME = "gpt-4o-mini"
EMBEDDING_MODEL_NAME = "text-embedding-3-small"
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200

### 가상 프롬프트 임베딩 생성 정의

아래 코드 블록은 각 텍스트 청크에 대한 가상 질문을 생성하고 검색을 위해 임베딩합니다.

- LLM이 입력 청크에서 핵심 질문을 추출합니다.
- 이러한 질문은 OpenAI 모델을 사용하여 임베딩됩니다.
- 이 함수는 원본 청크와 나중에 검색에 사용될 프롬프트 임베딩을 반환합니다.

깔끔한 출력을 위해 추가 줄바꿈이 제거되며, 필요시 정규식 파싱으로 목록 형식을 개선할 수 있습니다.

In [ ]:
def generate_hypothetical_prompt_embeddings(chunk_text: str):
    """
    단일 청크에 대해 여러 가상 질문을 생성하기 위해 LLM을 사용합니다.
    이러한 질문은 검색 중 청크의 '프록시'로 사용됩니다.

    매개변수:
    chunk_text (str): 청크의 텍스트 내용

    반환값:
    chunk_text (str): 청크의 텍스트 내용. 멀티스레딩을 쉽게 하기 위해 수행됩니다.
    hypothetical prompt embeddings (List[float]): 질문에서 생성된 임베딩 벡터 목록
    """
    llm = ChatOpenAI(temperature=0, model_name=LANGUAGE_MODEL_NAME)
    embedding_model = OpenAIEmbeddings(model=EMBEDDING_MODEL_NAME)

    question_gen_prompt = PromptTemplate.from_template(
        "Analyze the input text and generate essential questions that, when answered, \
        capture the main points of the text. Each question should be one line, \
        without numbering or prefixes.\n\n \
        Text:\n{chunk_text}\n\nQuestions:\n"
    )
    question_chain = question_gen_prompt | llm | StrOutputParser()

    # 응답에서 질문 파싱
    # 참고: 
    # - gpt4o는 \n\n로 질문을 분리하는 것을 선호하므로 하나의 \n을 제거합니다
    # - 프로덕션이나 ollama의 더 작은 모델을 사용하는 경우, 정규식을 사용하여
    # (비)순서 목록 같은 것을 파싱하는 것이 유익합니다
    # r"^\s*[\-\*\•]|\s*\d+\.\s*|\s*[a-zA-Z]\)\s*|\s*\(\d+\)\s*|\s*\([a-zA-Z]\)\s*|\s*\([ivxlcdm]+\)\s*"
    questions = question_chain.invoke({"chunk_text": chunk_text}).replace("\n\n", "\n").split("\n")
    
    return chunk_text, embedding_model.embed_documents(questions)


### FAISS 벡터 스토어 생성 및 채우기 정의

아래 코드 블록은 텍스트 청크를 병렬로 임베딩하여 FAISS 벡터 스토어를 구축합니다.

무슨 일이 일어나나요?
- 병렬 처리 – 임베딩 생성을 더 빠르게 하기 위해 스레딩을 사용합니다.
- FAISS 초기화 – 효율적인 유사도 검색을 위한 L2 인덱스를 설정합니다.
- 청크 임베딩 – 각 청크는 여러 번 저장되며, 생성된 각 질문 임베딩마다 한 번씩 저장됩니다.
- 메모리 내 저장 – 빠른 조회를 위해 InMemoryDocstore를 사용합니다.

이를 통해 효율적인 검색을 보장하며, 미리 계산된 질문 임베딩과의 쿼리 정렬을 개선합니다.

In [ ]:
def prepare_vector_store(chunks: List[str]):
    """
    텍스트 청크 목록에서 FAISS 벡터 스토어를 생성하고 채웁니다.

    이 함수는 텍스트 청크 목록을 병렬로 처리하여 각 청크에 대한
    가상 프롬프트 임베딩을 생성합니다.
    임베딩은 효율적인 유사도 검색을 위해 FAISS 인덱스에 저장됩니다.

    매개변수:
    chunks (List[str]): 임베딩되고 저장될 텍스트 청크 목록.

    반환값:
    FAISS: 임베딩된 텍스트 청크를 포함하는 FAISS 벡터 스토어.
    """

    # 벡터 길이를 확인하기 위해 초기화를 기다림
    vector_store = None  

    with ThreadPoolExecutor() as pool:  
        # 프롬프트 임베딩 생성 속도를 높이기 위해 스레딩 사용
        futures = [pool.submit(generate_hypothetical_prompt_embeddings, c) for c in chunks]
        
        # 완료되는 대로 임베딩 처리
        for f in tqdm(as_completed(futures), total=len(chunks)):  
            
            chunk, vectors = f.result()  # 처리된 청크와 그 임베딩 검색
            
            # 첫 번째 청크에서 FAISS 벡터 스토어 초기화
            if vector_store == None:  
                vector_store = FAISS(
                    embedding_function=OpenAIEmbeddings(model=EMBEDDING_MODEL_NAME),  # 임베딩 모델 정의
                    index=faiss.IndexFlatL2(len(vectors[0])),  # 유사도 검색을 위한 L2 인덱스 정의
                    docstore=InMemoryDocstore(),  # 메모리 내 문서 저장소 사용
                    index_to_docstore_id={}  # 인덱스-문서 매핑 유지
                )
            
            # 청크의 내용을 각 생성된 임베딩 벡터와 쌍으로 연결.
            # 각 청크는 여러 번 삽입되며, 각 프롬프트 벡터마다 한 번씩 삽입됨
            chunks_with_embedding_vectors = [(chunk.page_content, vec) for vec in vectors]
            
            # 스토어에 임베딩 추가
            vector_store.add_embeddings(chunks_with_embedding_vectors)  

    return vector_store  # 채워진 벡터 스토어 반환


### PDF를 FAISS 벡터 스토어로 인코딩

아래 코드 블록은 PDF 파일을 처리하고 검색을 위해 그 내용을 임베딩으로 저장합니다.

무슨 일이 일어나나요?
- PDF 로딩 – 문서에서 텍스트를 추출합니다.
- 청킹 – 더 나은 컨텍스트 유지를 위해 텍스트를 겹치는 세그먼트로 분할합니다.
- 전처리 – 임베딩 품질을 개선하기 위해 텍스트를 정리합니다.
- 벡터 스토어 생성 – 임베딩을 생성하고 검색을 위해 FAISS에 저장합니다.

In [ ]:
def encode_pdf(path, chunk_size=1000, chunk_overlap=200):
    """
    OpenAI 임베딩을 사용하여 PDF 책을 벡터 스토어로 인코딩합니다.

    인수:
        path: PDF 파일의 경로.
        chunk_size: 각 텍스트 청크의 원하는 크기.
        chunk_overlap: 연속된 청크 간의 겹침 양.

    반환값:
        인코딩된 책 내용을 포함하는 FAISS 벡터 스토어.
    """

    # PDF 문서 로드
    loader = PyPDFLoader(path)
    documents = loader.load()

    # 문서를 청크로 분할
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len
    )
    texts = text_splitter.split_documents(documents)
    cleaned_texts = replace_t_with_space(texts)

    vectorstore = prepare_vector_store(cleaned_texts)

    return vectorstore

### HyPE 벡터 스토어 생성

이제 PDF를 처리하고 그 임베딩을 저장합니다.
이 단계는 인코딩된 문서로 FAISS 벡터 스토어를 초기화합니다.

In [ ]:
# HyPE에서는 더 많은 정보로 인해 정밀도를 잃지 않으므로 청크 크기를 꽤 크게 설정할 수 있습니다.
# 프로덕션의 경우, 모델이 청크당 충분한 수의 질문을 생성하는 데 얼마나 철저한지 테스트하세요.
# 이것은 주로 정보 밀도에 따라 달라집니다.
chunks_vector_store = encode_pdf(PATH, chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)

100%|██████████| 97/97 [00:22<00:00,  4.40it/s]


### 데이터 확인 1단계: 청크 텍스트 확인

첫 번째 청크의 내용을 확인합니다.

In [ ]:
# 1. 청크 텍스트 확인 (첫 번째, 두 번째 청크)
import pandas as pd

# PDF를 다시 로드하여 청크 텍스트 확인 (또는 이미 생성된 cleaned_texts 사용)
loader = PyPDFLoader(PATH)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP, length_function=len
)
texts = text_splitter.split_documents(documents)
cleaned_texts = replace_t_with_space(texts)

# 첫 번째와 두 번째 청크 확인
chunk_indices = [0, 1]

for chunk_idx in chunk_indices:
    if chunk_idx < len(cleaned_texts):
        sample_chunk_text = cleaned_texts[chunk_idx].page_content
        
        print("=" * 80)
        print(f"청크 {chunk_idx}의 내용")
        print("=" * 80)
        print(f"청크 길이: {len(sample_chunk_text)} 문자")
        print("-" * 80)
        print("청크 내용 (처음 500자):")
        print("-" * 80)
        print(sample_chunk_text[:500] + "..." if len(sample_chunk_text) > 500 else sample_chunk_text)
        print("-" * 80)
        print()
    else:
        print(f"청크 {chunk_idx}는 존재하지 않습니다. (총 {len(cleaned_texts)}개 청크)")

print(f"총 청크 개수: {len(cleaned_texts)}")

### 데이터 확인 2단계: 청크에서 추출된 질문 확인

LLM이 청크 텍스트에서 생성한 질문들을 확인합니다.

In [ ]:
# 2. 청크에서 추출된 질문 확인 (첫 번째, 두 번째 청크)

# 질문 추출 함수 (확인용)
def get_questions_from_chunk(chunk_text: str):
    """청크에서 질문을 추출하여 반환합니다."""
    llm = ChatOpenAI(temperature=0, model_name=LANGUAGE_MODEL_NAME)
    
    question_gen_prompt = PromptTemplate.from_template(
        "Analyze the input text and generate essential questions that, when answered, \
        capture the main points of the text. Each question should be one line, \
        without numbering or prefixes.\n\n \
        Text:\n{chunk_text}\n\nQuestions:\n"
    )
    question_chain = question_gen_prompt | llm | StrOutputParser()
    
    response = question_chain.invoke({"chunk_text": chunk_text})
    questions = response.replace("\n\n", "\n").split("\n")
    # 빈 문자열 제거
    questions = [q.strip() for q in questions if q.strip()]
    return questions

# 첫 번째와 두 번째 청크에서 질문 추출
chunk_indices = [0, 1]
all_chunk_data = {}

for chunk_idx in chunk_indices:
    if chunk_idx < len(cleaned_texts):
        sample_chunk_text = cleaned_texts[chunk_idx].page_content
        sample_questions = get_questions_from_chunk(sample_chunk_text)
        all_chunk_data[chunk_idx] = {
            'chunk_text': sample_chunk_text,
            'questions': sample_questions
        }
        
        print("=" * 80)
        print(f"청크 {chunk_idx}에서 추출된 질문")
        print("=" * 80)
        print(f"추출된 질문 개수: {len(sample_questions)}")
        print("-" * 80)
        for idx, question in enumerate(sample_questions, 1):
            print(f"{idx}. {question}")
        print("-" * 80)
        print()
    else:
        print(f"청크 {chunk_idx}는 존재하지 않습니다. (총 {len(cleaned_texts)}개 청크)")

### 데이터 확인 3단계: 청크와 질문 임베딩 매핑 결과 확인

각 질문이 임베딩된 결과와 청크와의 매핑 관계를 확인합니다.

In [ ]:
# 3. 청크와 질문 임베딩 매핑 결과 확인 (첫 번째, 두 번째 청크)

embedding_model = OpenAIEmbeddings(model=EMBEDDING_MODEL_NAME)

chunk_indices = [0, 1]
all_mapping_results = []

for chunk_idx in chunk_indices:
    if chunk_idx < len(cleaned_texts):
        chunk_data = all_chunk_data[chunk_idx]
        sample_chunk_text = chunk_data['chunk_text']
        sample_questions = chunk_data['questions']
        
        # 각 질문을 임베딩
        question_embeddings = embedding_model.embed_documents(sample_questions)
        
        # 매핑 결과를 데이터프레임으로 정리
        mapping_df = pd.DataFrame({
            '질문_번호': range(1, len(sample_questions) + 1),
            '질문_텍스트': sample_questions,
            '임베딩_차원': [len(emb) for emb in question_embeddings],
            '임베딩_벡터_처음_5차원': [str(emb[:5]) for emb in question_embeddings]
        })
        
        all_mapping_results.append({
            'chunk_idx': chunk_idx,
            'df': mapping_df,
            'embeddings': question_embeddings
        })
        
        print("=" * 80)
        print(f"청크 {chunk_idx}와 질문 임베딩 매핑 결과")
        print("=" * 80)
        print(f"질문 개수: {len(sample_questions)}")
        print(f"임베딩 차원: {len(question_embeddings[0]) if question_embeddings else 0}")
        print("-" * 80)
        print(mapping_df.to_string(index=False))
        print("-" * 80)
        print()

### 데이터 확인 4단계: 청크와 질문이 함께 임베딩된 결과 확인

청크 텍스트 자체와 추출된 질문들이 모두 임베딩된 결과를 함께 확인합니다.

In [ ]:
# 4. 청크와 질문이 함께 임베딩된 결과 확인 (첫 번째, 두 번째 청크)

chunk_indices = [0, 1]

for chunk_idx in chunk_indices:
    if chunk_idx < len(cleaned_texts):
        chunk_data = all_chunk_data[chunk_idx]
        sample_chunk_text = chunk_data['chunk_text']
        sample_questions = chunk_data['questions']
        
        # 매핑 결과에서 임베딩 가져오기
        mapping_result = next((r for r in all_mapping_results if r['chunk_idx'] == chunk_idx), None)
        if mapping_result:
            question_embeddings = mapping_result['embeddings']
        else:
            question_embeddings = embedding_model.embed_documents(sample_questions)
        
        # 청크 텍스트 자체를 임베딩
        chunk_embedding = embedding_model.embed_documents([sample_chunk_text])[0]
        
        # 전체 매핑 정보를 포함한 데이터프레임
        full_mapping_data = []
        
        # 청크 정보 추가
        full_mapping_data.append({
            '타입': '청크',
            '번호': '-',
            '텍스트_미리보기': sample_chunk_text[:150] + "...",
            '전체_텍스트_길이': len(sample_chunk_text),
            '임베딩_차원': len(chunk_embedding),
            '임베딩_벡터_처음_5차원': str(chunk_embedding[:5])
        })
        
        # 질문 정보 추가
        for idx, (question, emb) in enumerate(zip(sample_questions, question_embeddings), 1):
            full_mapping_data.append({
                '타입': '질문',
                '번호': idx,
                '텍스트_미리보기': question,
                '전체_텍스트_길이': len(question),
                '임베딩_차원': len(emb),
                '임베딩_벡터_처음_5차원': str(emb[:5])
            })
        
        full_mapping_df = pd.DataFrame(full_mapping_data)
        
        print("=" * 80)
        print(f"청크 {chunk_idx}와 질문들이 모두 임베딩된 결과")
        print("=" * 80)
        print(full_mapping_df.to_string(index=False))
        print("-" * 80)
        
        # 요약 통계
        print(f"\n[청크 {chunk_idx} 임베딩 벡터 통계]")
        print(f"청크 임베딩 차원: {len(chunk_embedding)}")
        print(f"질문 임베딩 차원: {len(question_embeddings[0]) if question_embeddings else 0}")
        print(f"질문 개수: {len(question_embeddings)}")
        print(f"총 임베딩 벡터 개수: {1 + len(question_embeddings)}")
        print("-" * 80)
        print()

### 검색기 생성

이제 벡터 스토어에서 관련 청크를 가져오도록 검색기를 설정합니다.

쿼리 유사도를 기반으로 가장 관련성 높은 상위 `k=3` 개의 청크를 검색합니다.

In [79]:
chunks_query_retriever = chunks_vector_store.as_retriever(search_kwargs={"k": 3})

### 검색기 테스트

이제 샘플 쿼리를 사용하여 검색을 테스트합니다.

- 벡터 스토어를 쿼리하여 가장 관련성 높은 청크를 찾습니다.
- 잠재적으로 반복된 청크를 제거하기 위해 결과를 중복 제거합니다.
- 검사할 검색된 컨텍스트를 표시합니다.

이 단계는 검색기가 주어진 질문에 대해 의미 있고 다양한 정보를 반환하는지 확인합니다.

In [ ]:
# test_query = "What is the main cause of climate change?"
# context = retrieve_context_per_question(test_query, chunks_query_retriever)
# context = list(set(context))
# show_context(context)

test_query = "What is the main cause of climate change?"

# 벡터 스토어에서 직접 검색 (가장 안정적)
docs = chunks_vector_store.similarity_search(test_query, k=3)
context = [doc.page_content for doc in docs]

# 중복 제거
context = list(set(context))

# 컨텍스트 표시
show_context(context)

Context 1:
Most of these climate changes are attributed to very small variations in Earth's orbit that 
change the amount of solar energy our planet receives. During the Holocene epoch, which 
began at the end of the last ice age, human societies f lourished, but the industrial era has seen 
unprecedented changes.  
Modern Observations  
Modern scientific observations indicate a rapid increase in global temperatures, sea levels, 
and extreme weather events. The Intergovernmental Panel on Climate Change (IPCC) has 
documented these changes extensively. Ice core samples, tree rings, and ocean sediments 
provide a historical record that scientists use to understand past climate conditions and 
predict future trends. The evidence overwhelmingly shows that recent changes are primarily 
driven by human activities, particularly the emission of greenhou se gases.  
Chapter 2: Causes of Climate Change  
Greenhouse Gases


Context 2:
driven by human activities, particularly the emission of green

### 결과 평가

In [76]:
evaluate_rag(chunks_query_retriever)

{'questions': ['1. **Multiple Choice: Causes of Climate Change**',
  '   - What is the primary cause of the current climate change trend?',
  '     A) Solar radiation variations',
  '     B) Natural cycles of the Earth',
  '     C) Human activities, such as burning fossil fuels',
  '     D) Volcanic eruptions',
  '',
  '2. **True or False: Impact on Biodiversity**',
  '   - True or False: Climate change does not have any significant impact on the migration patterns and extinction rates of various species.',
  '',
  '3. **Short Answer: Mitigation Strategies**',
  '   - What are two effective strategies that can be implemented at a community level to mitigate the effects of climate change?',
  '',
  '4. **Matching: Climate Change Effects**',
  '   - Match the following effects of climate change (numbered) with their likely consequences (lettered).',
  '     1. Rising sea levels',
  '     2. Increased frequency of extreme weather events',
  '     3. Melting polar ice caps',
  '     4. Oce

![](https://europe-west1-rag-techniques-views-tracker.cloudfunctions.net/rag-techniques-tracker?notebook=all-rag-techniques--hype-hypothetical-prompt-embeddings)